In [ ]:
college_name = ''
course_name = ''
year = 19 

# Year should just be the last 2 digits of the joining year
# college_name & course_name must be EXACT strings from 
# college_codes.csv and course_codes.csv

# You can scrape college_codes.csv & course_codes.csv separately first
# Find the relevant college_name and course_name, paste it here before starting.

# A sample would look like this:

# college_name = "Faculty of Management Studies"
# course_name = "(P.G)- MASTER OF BUSINESS ADMINISTRATION (FULL TIME) (M.B.A)"
# year = 19

# DUScraper

This script scrapes the DU result website to extract student result data.  
I wrote this to practice scraping - the website is a decent challenge for a beginner as it contains a form that needs to be filled as well as a captcha that you need to pass in order to access the student results.  

All DU roll numbers look like this:

YYCC01CC1IND

Breaking it down into 4 components:  
__YY-CC0-CC1-IND__

- __YY__: Last 2 digits of joining year. For eg, 18.
- __CC0__: College code. This is found in the source code of the form for accessing the result, stored in college_codes_page link given below. These are generally 3 digit codes, like 036 or 027.
- __CC1__: Course code. This is found in the course_codes_page link given below. These are also generally 3 digit codes, like 510, 501 and so on.
- __IND__: Index number. These start at 001 and go up till 999(?) at most.

### Packages

In [ ]:
# To check for paths
import os

# Web crawler
from splinter import Browser #web crawler

# For handling data
import pandas as pd
import numpy as np 

# For plotting graphs
import matplotlib.pyplot as plt

from time import sleep
from math import ceil

### Parameters, links etc.

In [ ]:
browser_used = 'chrome'

In [ ]:
college_codes_page = 'http://durslt.du.ac.in/DURSLT_ND2020/Students/Combine_GradeCard.aspx'
course_codes_page = 'http://durslt.du.ac.in/DURSLT_ND2020/Students/List_Of_Declared_Results.aspx'
result_page = college_codes_page

In [ ]:
# The script will go to sleep for halt_time seconds after every 
# halt_after number of records are extracted. This is to avoid detection

halt_time = 30
halt_after = 15

In [ ]:
# Don't edit this unless the error msg changes on the DU website
error_msg = 'Sorry! no record found.' 

In [ ]:
# The script will allow p_errors_allowed back to back missing student records 
# to be checked before it concludes that the roll numbers have terminated.
# Edit according to common sense.
p_errors_allowed = 8

In [ ]:
start_at = 1 # Roll number 1
end_at = 20

# If you want the entire batch results, set end_at to 999
# Else, set it to a lower number for a sample.
# Note that it's possible there may be missing results in between!

# Note that the program will not check all between start_at and end_at. 
# In the middle, if it faces errors (missing results) repeatedly back to back more than 
# or equal to p_errors_allowed times, then it will consider that the roll numbers have 
# ended so end_at only defines an upper limit, 
# not the actual times the script will run to extract the data

### Pre-requisites (college codes & course codes)

In [ ]:
THIS_FOLDER = os.path.abspath('')

In [ ]:
prereq_data = True # flag var, will be set to False if the data is not there.

In [ ]:
if not os.path.exists( os.path.join(THIS_FOLDER, 'data/' )):
    os.makedirs(os.path.join(THIS_FOLDER, 'data/' ))

In [ ]:
if not os.path.exists( os.path.join(THIS_FOLDER, 'prereq/' )):
    os.makedirs(os.path.join(THIS_FOLDER, 'prereq/' ))

### College codes

__Note:__ All college codes will be scraped, regardless of whether their results are out yet or not

In [ ]:
codes = pd.DataFrame()

try:
    codes = pd.read_excel( os.path.join(THIS_FOLDER, 'prereq/college_codes.xls'))
    print("College codes loaded!")
    
except:
    print("College codes not found!")
    print("Scraping college code data...")
    
    browser = Browser(browser_used)
    browser.visit(college_codes_page)

    selection = browser.find_by_id('ddlcollege')
    options = selection.find_by_tag('option')
    
    college_codes=dict()
    for i in range(1, len(options)):
        college_codes[ options[i].value] = options[i].text

    col_codes_data = pd.DataFrame(list(college_codes.items()), index=college_codes.keys() )
    col_codes_data.columns=['code', 'name']
    col_codes_data.set_index(['code'], inplace=True)

    browser.quit()
    print("College codes scraped!")
    
    col_codes_data.to_excel(os.path.join(THIS_FOLDER, 'prereq/college_codes.xls'))
    codes = pd.read_excel(os.path.join(THIS_FOLDER, 'prereq/college_codes.xls'))
    print("College codes loaded!")

### Course Codes

__Note:__ Only those course codes will be scraped whose results have been declared yet on the course_codes_page! If you're trying to scrape the results for a course that just got declared, you need to delete course_codes.xls first and then run the cell below to get the updated list of course_codes!

In [ ]:
course_codes = pd.DataFrame()

try:
    course_codes = pd.read_excel(os.path.join(THIS_FOLDER, 'prereq/course_codes.xls'))
    print("Course codes loaded!")

except:
    print("Course codes not found!")
    print("Scraping course code data...")
    
    browser = Browser(browser_used)
    browser.visit(course_codes_page)
    
    data_result = browser.find_by_xpath("//table[@id='gvshow_Reg']//tbody//tr")

    data = list()
    header = list()
    
    th = data_result[0].find_by_tag('th')
    
    for i in range(len(th)):
        header.append(th[i].text)
        
    
    for i in range(1, len(data_result)):
        r_data = list()
        td = data_result[i].find_by_tag('td')
        for a in range(len(td)):
            r_data.append(td[a].text)
            
        data.append(r_data)
    
    tempData = pd.DataFrame(data, columns = header)
    
    course_codes = tempData[['Course Code', 'Course Name']].drop_duplicates().set_index('Course Code')
    browser.quit()
    print("Course codes scraped!")

    
    course_codes.to_excel(os.path.join(THIS_FOLDER, 'prereq/course_codes.xls'))
    course_codes = pd.read_excel(os.path.join(THIS_FOLDER, 'prereq/course_codes.xls'))
    print("Course codes loaded!")

## Scraping

In [ ]:
college_code = codes[ codes['name'] == college_name]['code'].values[0]
course_code = course_codes[ course_codes['Course Name'] == course_name]['Course Code'].values[0]

In [ ]:
students = [f"{i:03d}" for i in range(start_at, end_at + 1)] # 001,002,003 and so on...
base_roll_no = f"{year}{college_code}{course_code}"

In [ ]:
which_dir = f"data/{college_code}_{college_name}/{course_code}_{course_name}/20{year}"
base_dir = os.path.join(THIS_FOLDER, which_dir)

In [ ]:
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

In [ ]:
# Extracting raw data
first = True # flag var, set to False after first result is found
header = list()
raw_data = pd.DataFrame()
count = 0

# The scraping won't begin if raw_data.xls exists in the base_dir
execute = not (os.path.exists(os.path.join(base_dir, 'raw_data.xls')))

if(execute):
    p_errors=0

    browser = Browser(browser_used)

    for w in students:

        roll_num = base_roll_no + w

        browser.visit(result_page) # visit webpage
        browser.find_by_id('ddlcollege').select(college_code) # select college

        src = browser.evaluate_script("document.getElementById('imgCaptcha').src")
        captcha = src[ (src.find("Code=") + 5) : (src.find("Code") + 11) ]
        # The captcha is a 6 digit number given in the link of the image itself
        
        browser.fill("txtcaptcha", captcha)

        browser.find_by_id('txtrollno').fill(str(roll_num)) # fill the roll number
        browser.find_by_id("btnsearch").click() # go

        error = False
        try:
            error = (browser.find_by_id("lblmsg").first.text == error_msg)
        except:
            print(f"[{roll_num}] Processing.")

        if(error):
            print(f"[{roll_num}] Record not found.")
            p_errors += 1
            if(p_errors == p_errors_allowed):
                print(f"[NOTE] Persistent errors have reached upper limit of p_errors_allowed = {p_errors_allowed}")
                print("[END] Terminating data collection...")
                break

        else:
            p_errors = 0 # p_errors reset if a result is found!
            
            data_rollno = browser.find_by_id("lblrollno").first.text
            data_name = browser.find_by_id("lblname").first.text
            data_result = browser.find_by_xpath("//table[@id='gvshow']//tbody//tr")

            data = list()

            if(first):
                header = list()
                th = data_result[0].find_by_tag('th')
                for i in range (len(th)):
                    header.append(th[i].text)

            for i in range(1, len(data_result)):
                r_data = list()
                td = data_result[i].find_by_tag('td')
                for a in range(len(td)):
                    r_data.append(td[a].text)
                    
                data.append(r_data)

            tempData = pd.DataFrame(data, columns=header)
            tempData.insert(0, "RollNo", [str(data_rollno) for i in range(0, len(data_result) - 1)])
            tempData.insert(1, "Name", [str(data_name) for i in range(0, len(data_result) - 1)])
            # len(data) - 1 because the first row is header row

            raw_data = tempData if first else pd.concat([raw_data, tempData])
            first = False

            count += 1
            if(count % halt_after == 0):
                sleep(halt_time)

    # Done scraping
    raw_data.set_index(["RollNo", "Name"])
    browser.quit()
    print("Scraping over!")
    
    if(count > 0):
        raw_data.to_excel(os.path.join(base_dir, 'raw_data.xls'), sheet_name='raw')
        print("Raw data exported")
    

### Processing

DU results are presented differently depending on the course. The section of the script below has been tested for some undergrad data (in particular, 3 year CBCS courses) where data for each paper is presented in terms of a "Grade Point" between 0 to 10. Using this, the below section generates the following:
- An Excel file which contains paperwise grades
- An Excel file which contains the GPA
- A PDF file which contains the grade distributions for all the papers in the raw data

__Note:__ There are certain courses for which the data is presented differently, so the section below won't work on that data without transforming it first.

In [ ]:
raw_data = pd.read_excel(os.path.join(base_dir, 'raw_data.xlsx'))

### Paperwise Data

In [ ]:
papers = list(np.unique( raw_data['Paper Name']))
raw_data.set_index(['RollNo'], inplace=True)
datasets = dict()
sheet_count=0

with pd.ExcelWriter(os.path.join(base_dir, 'paperwise.xls')) as writer:
    for i in papers:
        temp_data = raw_data[ raw_data['Paper Name'] == i][['Name', 'Grade Point']]
        temp_data.columns = ['Name', i]
        datasets[i] = temp_data
        datasets[i].to_excel(writer, sheet_name=str(sheet_count))
        sheet_count += 1
        
print("Paperwise data segregated!")

### Paperwise Distributions

In [ ]:
cols = 2
rows = ceil(len(papers) / 2)

plt.style.use('ggplot')
fig, ax = plt.subplots(rows, cols,figsize=(15, 5 * ceil(len(papers) / 2)))

plot_count = 0
workingData = 0

for i in range(rows):
    for j in range(cols):
        if(plot_count == len(papers)):
            break
        
        workingData = raw_data[ raw_data['Paper Name'] == papers[plot_count] ]['Grade Point']
        ax[i, j].hist(workingData, bins=range(0,12), range=(0, 13))
        ax[i,j].spines["top"].set_visible(False)
        ax[i,j].spines["right"].set_visible(False)
        ax[i,j].set_xlim(0, 12)
        ax[i,j].set_ylim(0,  workingData.value_counts().max()+2)
        ax[i,j].set_title(papers[plot_count])
        ax[i,j].set_ylabel('Students')
        ax[i,j].set_xlabel('Marks')
        plot_count+=1

plt.tight_layout()

fig.savefig( os.path.join(base_dir, 'paperwise_distr.pdf'), bbox_inches='tight')

### GPA Data

In [ ]:
n_raw_data = raw_data.reset_index(drop=False)
gpa_data = pd.DataFrame(columns=['RollNo', 'Name', 'GPA'])

for k in students:
    roll_no = base_roll_no + k
    tmp_data = n_raw_data[ n_raw_data['RollNo'] == int(roll_no) ]
    tmp_data.reset_index(drop = True, inplace = True)
    
    if(tmp_data.count().sum() != 0):
        tmp_data = tmp_data.assign(wgrade = (tmp_data['Grade Point'] * tmp_data['Credit'] ))
        name=tmp_data['Name'][0]
        GPA= (tmp_data['wgrade'].sum()) / (tmp_data['Credit'].sum())

        gpa_data = gpa_data.append( {'RollNo':roll_no, 'Name':name, 'GPA':GPA}, ignore_index=True)


with pd.ExcelWriter(os.path.join(base_dir, 'gpa.xls')) as writer:
    gpa_data.to_excel(writer, sheet_name='unsorted')
    gpa_data.sort_values(by = ['GPA'], ascending=False).reset_index(drop=True).to_excel(writer, sheet_name='sorted')
    
print("GPA calculated!")